## Hamiltonian Simulation Observable Benchmark Tests - WIP

This notebook is being used to flesh out the details of integrating the HamLib utility module, the HamLib kernel function, and the newly developed functions for generating groups of commuting terms.

From any specific Hamiltonian, we create a base evolution circuit with 0 - N Trotter steps, initialize the circuit, and append the necessary basis rotations before measuring and computing the values of energy and other related observables.

At a later point, the execution code and plotting code will also be integrated into the benchmark framework. For now, however, the focus is on developing the code necessary to evaluate the performance of observable execution using various optimization options.


### Setup Dependencies and Initialize Modules

This cell is used to pull in dependencies from standard libraries, the Hamlib helper functions, and the newly created observables module (in WIP).
Initialize the modules so a re-run from the beginning is repeatable.

In [1]:
import numpy as np
from math import sin, cos, pi
import time

# Import Qiskit and Qiskit Pauli operator classes
from qiskit.quantum_info import Pauli, SparsePauliOp
from qiskit import QuantumCircuit

# Initialize simulator backend
from qiskit_aer import Aer
backend = Aer.get_backend('qasm_simulator')

# Import Hamlib helper functions
import hamlib_simulation_kernel, hamlib_utils

# Restore default parameters in HamLib kernel module
hamlib_simulation_kernel.initialize()

# Import Observable helper functions
import sys
sys.path.insert(1, "WIP_benchmarks")
import observables


### Configure Options for Execution

In [2]:
# Qubit width of the Hamiltonian
num_qubits = 4

# Parameters of Trotterized simulation
K = 1
t = 0.05

# for debugging Hamlib helper functions
hamlib_simulation_kernel.verbose = True
hamlib_utils.verbose = False

# for debugging observables module
observables.verbose_circuits = False


### Specify a Hamiltonian for Testing
Choose a Hamiltonian from HamLib or create a custom Hamiltonian


In [3]:
# create the HamLib Simulation kernel, and the associated Hamiltonian operator

hamlib_simulation_kernel.initialize()

# Specify the desired Hamiltonian from HamLib 
# Set the hamiltonian_name: 'TFIM', 'Fermi-Hubbard-1D', 'Bose-Hubbard-1D', 'Heisenberg' or 'Max3Sat'
# If no name, it means we use an explicitly specified Hamiltonian
hamiltonian_name = 'TFIM'

########### HamLib Hamiltonians

if hamiltonian_name == 'TFIM':
    hamlib_simulation_kernel.global_pbc_val = 'pbc'
    hamlib_simulation_kernel.global_h = 2 

if hamiltonian_name == 'Heisenberg':
    hamlib_simulation_kernel.global_pbc_val = 'pbc'
    hamlib_simulation_kernel.global_h = 2
    
if hamiltonian_name == 'Fermi-Hubbard-1D':
    hamlib_simulation_kernel.global_pbc_val = 'pbc'
    hamlib_simulation_kernel.global_U = 12
    hamlib_simulation_kernel.global_enc = 'bk'

if hamiltonian_name == 'Bose-Hubbard-1D':
    hamlib_simulation_kernel.global_pbc_val = 'nonpbc'
    hamlib_simulation_kernel.global_U = 10
    hamlib_simulation_kernel.global_enc = 'gray'

if hamiltonian_name == 'Max3Sat':
    hamlib_simulation_kernel.global_ratio = 2
    hamlib_simulation_kernel.global_rinst = '02'
    
########### Explicit Hamiltonians

# classical simple Ising is ZZ
# TFIM ZZ + X  is transverse field
# + longitudinal field -> ZZ, X, and Z
def get_ising_hamiltonian(L, J, h, alpha=0):

    # List of Hamiltonian terms as 3-tuples containing
    # (1) the Pauli string,
    # (2) the qubit indices corresponding to the Pauli string,
    # (3) the coefficient.
    ZZ_tuples = [("ZZ", [i, i + 1], -J) for i in range(0, L - 1)]
    Z_tuples = [("Z", [i], -h * sin(alpha)) for i in range(0, L)]
    X_tuples = [("X", [i], -h * cos(alpha)) for i in range(0, L)]

    # We create the Hamiltonian as a SparsePauliOp, via the method
    # `from_sparse_list`, and multiply by the interaction term.
    hamiltonian = SparsePauliOp.from_sparse_list([*ZZ_tuples, *Z_tuples, *X_tuples], num_qubits=L)
    return hamiltonian.simplify()

# extract Hamiltonian from HamLib
if hamiltonian_name != '':
    
    params = hamlib_simulation_kernel.get_params_from_globals(hamiltonian_name)

    # load the HamLib file for the given hamiltonian name
    hamlib_utils.load_from_file(filename=hamiltonian_name)

    """
    datasets = hamlib_utils.find_dataset_for_params(num_qubits=num_qubits, params=params)
    print(f"... datasets = \n{datasets}")
    """

    # return a sparse Pauli list of terms queried from the open HamLib file
    parsed_pauli_list = hamlib_utils.get_hamlib_sparsepaulilist(num_qubits=num_qubits, params=params)
    print(f"... parsed_pauli_list = \n{parsed_pauli_list}")
    
    # convert the SparsePauliList to a SparsePauliOp object
    ham_op = hamlib_simulation_kernel.sparse_pauliop(parsed_pauli_list, num_qubits)
    print(f"... ham_op = \n{ham_op}")
    print("")
    
    # read the HamLib file content for the specified Hamiltonian and return a SparsePauliOp
    # Replaced with above code; above code may get encasupated in the below function later.
    """
    ham_op, _ = hamlib_simulation_kernel.get_hamlib_sparsepauliop(hamiltonian_name, num_qubits)
    print(f"... ham_op = \n{ham_op}")
    print("")
    """
    
    # convert SparsePauliOp to list form
    ham_terms = ham_op.to_list()
    
# create explicit Hamiltonian
else:
    H = get_ising_hamiltonian(L=num_qubits, J=0.2, h=1.2, alpha=pi / 8)
    H_terms = H.to_list()
    
    # hamiltonian = H_terms
    # ham_terms = hamiltonian
    ham_terms = H_terms

print(f"... Hamiltonian name = {hamiltonian_name}")
print(ham_terms)
print("")


... get_params_from_globals() ==> {'h': 2, '1D-grid': 'pbc'}
... parsed_pauli_list = 
([({0: 'Z', 1: 'Z'}, (1+0j)), ({0: 'Z', 3: 'Z'}, (1+0j)), ({1: 'Z', 2: 'Z'}, (1+0j)), ({3: 'Z', 2: 'Z'}, (1+0j)), ({0: 'X'}, (2+0j)), ({1: 'X'}, (2+0j)), ({3: 'X'}, (2+0j)), ({2: 'X'}, (2+0j))], 'graph-1D-grid-pbc-qubitnodes_Lx-4_h-2')


ValueError: too many values to unpack (expected 2)

### Create a Hamiltonian Simulation Circuit
Initialize the circuit to a known initial state, append 'K' Trotter steps for total time 't', followed by basis rotation gates.

In [ ]:
# create the HamLib Simulation kernel, from the current Hamiltonian.
# We use a different method for kernel creation with HamLib than with custom Hamiltonians

########### create Trotterized evolution circuit for HamLib Hamiltonians

# create Hamiltonian evolution circuit from HamLib
if hamiltonian_name != '':
    init_state = "checkerboard"

    qc, bitstring = hamlib_simulation_kernel.HamiltonianSimulation(
        num_qubits=num_qubits, 
        ham_op=ham_op,
        K=K, t=t,
        init_state = init_state,
        append_measurements = False,
        method = 1, 
    )    
    
########### create Trotterized evolution circuit for custom Hamiltonian
else:
    #qc = None
    qc = QuantumCircuit(num_qubits)

print(f"... Trotterized Circuit, K={K}, t={t}")
if num_qubits < 11:
    if hamiltonian_name != '':
        hamlib_simulation_kernel.kernel_draw(qc, method=1)
    else:
        print(qc)
else:
    print("  ... circuit is too large to draw.")

print("")


### Compute Energy of the Hamiltonian - Unoptimized

Here, we convert the Hamiltonian to a set of circuits, using commuting groups if specified,
and execute the circuits in order to compute the expectation value 

In [ ]:
# Flag to control optimize by use of commuting groups
use_commuting_groups = False

N = 3
print(f"\n************ Compute energy of the Hamiltonian {N} times")

# Calculate the total energy
for i in range(N):
        
    # Estimate expectation for the Ham terms using the circuit qc
    total_energy, term_contributions = observables.estimate_expectation_plus(backend, qc, ham_terms, 
                                             use_commuting_groups=use_commuting_groups, num_shots=10000)
    print(f"Total Energy: {total_energy}")
    print(f"Term Contributions: {term_contributions}")
    
print("")

### Compute Energy of the Hamiltonian - Optimized

Here, we convert the Hamiltonian to a set of circuits, using commuting groups if specified,
and execute the circuits in order to compute the expectation value 

In [ ]:
# Flag to control optimize by use of commuting groups
use_commuting_groups = True

N = 3
print(f"\n************ Compute energy of the Hamiltonian {N} times")

# Calculate the total energy
for i in range(N):
        
    # Estimate expectation for the Ham terms using the circuit qc
    total_energy, term_contributions = observables.estimate_expectation_plus(backend, qc, ham_terms, 
                                             use_commuting_groups=use_commuting_groups, num_shots=10000)
    print(f"Total Energy: {total_energy}")
    print(f"Term Contributions: {term_contributions}")
    
print("")

### Compute expectation value of other observables.
Note that this only works for the Ising model.

In [ ]:
# Define additional Hamiltonian terms for other Ising observables 
H_terms_spin_correlation = observables.swap_pauli_list([(0.2,'IIIIZZ'), (0.2,'IIIZZI'), (0.2,'IIZZII'), (0.2,'IZZIII'), (0.2,'ZZIIII')])
H_terms_magnetization = observables.swap_pauli_list([(1,'IIIIIZ'), (1,'IIIIZI'), (1,'IIIZII'), (1,'IIZIII'), (1,'IZIIII'), (1, 'ZIIIII')])

spin_correlation = observables.calculate_expectation_from_contributions(H_terms_spin_correlation, term_contributions)
print(spin_correlation)

magnetization = observables.calculate_expectation_from_contributions(H_terms_magnetization, term_contributions)
print(magnetization)
